# Visualization of the convolution operation

This demo is supposed to visualize the [convolution](https://en.wikipedia.org/wiki/Convolution) operation. 

This demo is written by [Markus Nölle](https://www.htw-berlin.de/hochschule/personen/person/?eid=9586) for a basic course on signals and systems hold at the [university of applied siences, Berlin](https://www.htw-berlin.de/).

## Import libraries and define defaults

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import ipywidgets as widgets

plt.style.use('noelle.mplstyle')

## Define helper functions

In [2]:
def generate_signal(t, type):    
     
    if type == 'rect':
        sig = np.zeros(t.size)        
        sig[np.abs(t) <= 1/2]= 1
    
    elif type == 'exp':
        # decay rate
        alpha = -5        
        sig = np.exp(alpha * t)
        sig[t<0] = 0
        
    elif type == 'dirac':
        sig = np.zeros(t.size)
        sig[int(np.size(t)/2)] = 1
    
    return sig

In [3]:
def plot_signals(t, sr, x, y, tau, dur):
    
    # reverse
    y_flip = y[::-1]
    y_flip_shift = np.roll(y_flip, int(tau*sr))
    
    overlap_surface = x * y_flip_shift
    
    idx = np.argmin(np.abs(t-tau))
    
    z = signal.convolve(x, y, mode='same')    
    
    # prepare plots
    n_row = 2
    n_col = 3
    fig_size = [i*j for i,j in zip(plt.rcParams['figure.figsize'], [n_col, n_row])]
    fig = plt.figure(figsize=fig_size)
    
    # plot time signal    
    ax = fig.add_subplot(2, 3, 1)
    ax.plot(t, x, 'C0')
    ax.set_xlim(-dur/2, dur/2)
    ax.set_xlabel('time / s')
    ax.set_ylabel(r'x($\tau$) / a.u.') 
    
    ax = fig.add_subplot(2, 3, 2)
    ax.plot(t, y, 'C3')
    ax.set_xlim(-dur/2, dur/2)
    ax.set_xlabel('time / s')
    ax.set_ylabel(r'y($\tau$) / a.u.')
    
    ax = fig.add_subplot(2, 3, 3)
    ax.plot(t, y_flip_shift, 'C1')
    ax.set_xlim(-dur/2, dur/2)
    ax.set_xlabel('time / s')
    ax.set_ylabel(r'y(t-$\tau$) / a.u.')
    
    ax = fig.add_subplot(2, 3, 4)
    ax.plot(t, x, 'C0', t, y_flip_shift, 'C1')
    poly = ax.fill_between(t, np.zeros_like(overlap_surface), overlap_surface, color='C2', alpha=0.5)
    ax.set_xlim(-dur/2, dur/2)
    ax.set_xlabel('time / s')
    ax.set_ylabel('amplitude / a.u.')
    ax.legend([r'x(t)',r'y(t-$\tau$)'])
    y_lim = ax.get_ylim()
    
    ax = fig.add_subplot(2, 3, 5)
    ax.plot(t, overlap_surface, 'black')
    poly = ax.fill_between(t, np.zeros_like(overlap_surface), overlap_surface, color='C2', alpha=0.5)
    ax.set_xlim(-dur/2, dur/2)
    ax.set_xlabel('time / s')
    ax.set_ylim(y_lim)
    ax.set_ylabel(r'x(t)*y(t-$\tau$) / a.u.')    
    
    ax = fig.add_subplot(2, 3, 6)
    ax.plot(t, z, 'C2')
    line = ax.plot(t[idx], z[idx], marker='o', markersize=12, markerfacecolor='red')
    ax.set_xlim(-dur/2, dur/2)
    ax.set_xlabel('time / s')
    ax.set_ylabel('z(t) / a.u.')
    
    plt.tight_layout()
    plt.show()

In [4]:
def visualize_convolution(tau, signal1_type, signal2_type):   
    
    sr = 1000 # sample rate
    dur = 5 # duration of signals in s    
    t = np.arange(-dur/2, dur/2, 1/sr)
    
    x = generate_signal(t, signal1_type)
    y = generate_signal(t, signal2_type)
    
    plot_signals(t, sr, x, y, tau, dur)

## Actual visualization

This interactive demo lets you inspect the steps of the [convolution](https://en.wikipedia.org/wiki/Convolution) operation, formal given as 
\begin{eqnarray}
z(t)&=&x(t)\ast y(t)\\
&=&\int_{-\infty}^{\infty}x(\tau)\cdot y(t-\tau) d\tau.
\end{eqnarray}

This formal definition of the convolution operation can visually be interpreted as following steps:
* flip of the second signal around the origin $\left(y(-\tau)\right)$
* temporal shift of the flipped signal by $-t$ $\left(y(t-\tau)\right)$
* multiplication of the first signal with the flipped and shifted signal $\left(x(\tau)\cdot y(t-\tau)\right)$
* integration of the result of this multiplication $\left(\int_{-\infty}^{\infty}x(\tau)\cdot y(t-\tau) d\tau\right)$.

The above steps are depicted in the following graphs and can be visualized for different types of signals, namely a [rectangular pulse](https://en.wikipedia.org/wiki/Rectangular_function), an [exponential decay](https://en.wikipedia.org/wiki/Exponential_function) and a [Dirac impulse](https://en.wikipedia.org/wiki/Dirac_delta_function). 

Please note, that the properties of the individual functions (like width, decay rate, shifts, etc.) cannot be changed via sliders. However, feel free to experiment with different functions and properties by simply changing the code in the `generate_signal` function above.

In [5]:
w_tau = widgets.FloatSlider(min=-2., max=2., step=0.1, value=0.2, continuous_update=False, description='t:')
w_signal1 = widgets.RadioButtons(options=['rect', 'exp', 'dirac'], value='rect', description='Signal 1:', disabled=False)
w_signal2 = widgets.RadioButtons(options=['rect', 'exp', 'dirac'], value='rect', description='Signal 2:', disabled=False)

ui = widgets.HBox([w_tau, w_signal1, w_signal2])
out = widgets.interactive_output(visualize_convolution, {'tau':w_tau, 'signal1_type':w_signal1, 'signal2_type':w_signal2})
out.layout.height = '600px'
# display all
display(ui, out)

Output(layout=Layout(height='600px'))